In [1]:
import os

import torch
import torch.nn as nn
import torch_burn as tb
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
tb.__version__

'0.0.3.11'

In [3]:
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 1000
sample_dir = 'samples'

In [4]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [5]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [6]:
ds = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

In [7]:
data_loader = torch.utils.data.DataLoader(dataset=ds,
                                          batch_size=batch_size, 
                                          shuffle=True,
                                          num_workers=6)

In [8]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [9]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [10]:
D = D.cuda()
G = G.cuda()

In [11]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [12]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [13]:
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).cuda()
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).cuda()
        fake_labels = torch.zeros(batch_size, 1).cuda()

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).cuda()
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).cuda()
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [10/60], d_loss: 0.8522, g_loss: 0.5994, D(x): 0.95, D(G(z)): 0.55
Epoch [0/200], Step [20/60], d_loss: 0.9598, g_loss: 0.5233, D(x): 0.99, D(G(z)): 0.61
Epoch [0/200], Step [30/60], d_loss: 0.7654, g_loss: 0.7754, D(x): 0.95, D(G(z)): 0.51
Epoch [0/200], Step [40/60], d_loss: 0.8187, g_loss: 0.8530, D(x): 0.82, D(G(z)): 0.46
Epoch [0/200], Step [50/60], d_loss: 1.4748, g_loss: 0.5063, D(x): 0.71, D(G(z)): 0.67
Epoch [0/200], Step [60/60], d_loss: 0.9798, g_loss: 0.9109, D(x): 0.77, D(G(z)): 0.51
Epoch [1/200], Step [10/60], d_loss: 0.3390, g_loss: 1.8963, D(x): 0.92, D(G(z)): 0.23
Epoch [1/200], Step [20/60], d_loss: 0.3208, g_loss: 2.0294, D(x): 0.93, D(G(z)): 0.22
Epoch [1/200], Step [30/60], d_loss: 0.4999, g_loss: 1.7167, D(x): 0.90, D(G(z)): 0.31
Epoch [1/200], Step [40/60], d_loss: 0.4091, g_loss: 1.9480, D(x): 0.91, D(G(z)): 0.26
Epoch [1/200], Step [50/60], d_loss: 0.3198, g_loss: 2.2198, D(x): 0.93, D(G(z)): 0.21
Epoch [1/200], Step [60/60], d_loss: 0.2419

In [ ]:
class GANTrainer(tb.Trainer):
    def forward(self, data, is_train: bool):
        D, G = self.model
        optimD, optimG = self.optim

        image = data[0].flatten(1)
        batch_size = image.shape[0]
        real_label = data[1].reshape(batch_size, 1).type(torch.float32)  # 예제에서는 왜 real label을 안 썼지?
        fake_labels = torch.zeros(batch_size, 1)
        image, real_label, fake_labels = self.cuda(image, real_label, fake_labels)

        # Train D
        real_pred = D(image)
        d_loss_real = self.criterion(real_pred, real_label)

        z = torch.randn(batch_size, latent_size)
        z = self.cuda(z)
        fake_images = G(z)
        fake_score = D(fake_images)
        d_loss_fake = self.criterion(fake_score, fake_labels)

        # Backward D
        d_loss = d_loss_real + d_loss_fake
        if is_train:
            optimD.zero_grad()
            d_loss.backward()
            optimD.step()

        # Train G
        z = torch.randn(batch_size, latent_size)
        z = self.cuda(z)
        fake_images = G(z)
        fake_labels = D(fake_images)
        g_loss = self.criterion(fake_labels, real_label)

        if is_train:
            optimG.zero_grad()
            g_loss.backward()
            optimG.step()

        return image, real_pred, real_label, d_loss_real.item(), d_loss_fake.item(), g_loss.item()

    def backward(self, x: torch.Tensor, pred: torch.Tensor, y: torch.Tensor, is_train: bool):
        pass

    def loop(self, batch_idx, data, is_train: bool, losses: dict, logs: dict):
        x, pred, y, d_loss_real, d_loss_fake, g_loss = self.forward(data, is_train)

        name1 = 'd_loss_real'
        name2 = 'd_loss_fake'
        name3 = 'g_loss'
        if not is_train:
            name1 = 'val_' + name1
            name2 = 'val_' + name2
            name3 = 'val_' + name3

        losses[name1] = d_loss_real
        losses[name2] = d_loss_fake
        losses[name3] = g_loss
        if name1 not in logs:
            logs[name1] = d_loss_real
            logs[name2] = d_loss_fake
            logs[name3] = g_loss
        else:
            logs[name1] = self._ignition_mean(logs[name1], d_loss_real, batch_idx)
            logs[name2] = self._ignition_mean(logs[name2], d_loss_fake, batch_idx)
            logs[name3] = self._ignition_mean(logs[name3], g_loss, batch_idx)

        return x, pred, y

In [ ]:
trainer = GANTrainer(nn.ModuleList((D, G)), criterion, (d_optimizer, g_optimizer), ncols=200)

In [ ]:
trainer.fit(ds, train_valid_split=0.2, num_epochs=num_epochs,
            batch_size=batch_size, pin_memory=True)